<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/cloud_statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
DBLink = "https://cloud-project-5adfc-default-rtdb.europe-west1.firebasedatabase.app/"

#Read sensor data part

In [2]:
!pip install paho-mqtt
!pip install firebase

In [3]:
#Imports
import paho.mqtt.client as mqtt
import json
import threading
import time
from firebase import firebase
from datetime import datetime

FBconn = firebase.FirebaseApplication(DBLink, None)

In [24]:
last_print_time_indoor = 0
last_print_time_outdoor = 0

# Callback function for when a message is received
def indoor_on_message(client, userdata, msg):
  global last_print_time_indoor, indoor_flag, indoor_last_data
  if time.time() - last_print_time_indoor < 60:
    return

  try:
    last_print_time_indoor = time.time()
    data = json.loads(msg.payload.decode())
    # Extract values with lowercase field names
    temperature = data.get("Temperature", "N/A")
    humidity = data.get("Humidity", "N/A")
    pressure = data.get("Pressure", "N/A")
    distance = data.get("Distance", "N/A")
    print(f"Indoor: Temperature: {temperature}°C, Humidity: {humidity}%, Pressure: {pressure} hPa, Distance: {distance} mm")

    # Prepare data for Firebase
    indoor_sensor_data = {
        "Indoor Temperature": temperature,
        "Indoor Humidity": humidity,
        "Indoor Pressure": pressure,
        "Indoor Distance": distance
    }

    # Generate a unique key based on timestamp
    timestamp_key = datetime.now().strftime('%Y-%m-%d %H:%M:00')

    # Save the data to Firebase
    FBconn.put('/sensor_readings/indoor', timestamp_key, indoor_sensor_data)

  except json.JSONDecodeError:
    print("Received invalid JSON data")

# Callback function for when a message is received
def outdoor_on_message(client, userdata, msg):
  global last_print_time_outdoor
  if time.time() - last_print_time_outdoor < 60:
    return

  try:
    last_print_time_outdoor = time.time()
    data = json.loads(msg.payload.decode())
    # Extract values with lowercase field names
    temperature = data.get("Temperature", "N/A")
    humidity = data.get("Humidity", "N/A")
    dlight = data.get("Dlight", "N/A")
    print(f"Outdoor: Temperature: {temperature}°C, Humidity: {humidity}%, Dlight: {dlight} Lux")

    # Prepare data for Firebase
    outdoor_sensor_data = {
        "Outdoor Temperature": temperature,
        "Outdoor Humidity": humidity,
        "Outdoor Dlight": dlight
    }

    # Generate a unique key based on timestamp
    timestamp_key = datetime.now().strftime('%Y-%m-%d %H:%M:00')

    # Save the data to Firebase
    FBconn.put('/sensor_readings/outdoor', timestamp_key, outdoor_sensor_data)


  except json.JSONDecodeError:
    print("Received invalid JSON data")


In [ ]:
# Get the data and print

# MQTT setup
broker = "test.mosquitto.org"
topic_indoor = "braude/D106/indoor"
topic_outdoor = "braude/D106/outdoor"

client_indoor = mqtt.Client()
client_indoor.on_message = indoor_on_message

client_indoor.connect(broker, 1883, 60)
client_indoor.subscribe(topic_indoor)

client_outdoor = mqtt.Client()
client_outdoor.on_message = outdoor_on_message

client_outdoor.connect(broker, 1883, 60)
client_outdoor.subscribe(topic_outdoor)

# Create and start threads for each client
indoor_thread = threading.Thread(target=client_indoor.loop_forever)
outdoor_thread = threading.Thread(target=client_outdoor.loop_forever)

indoor_thread.start()
outdoor_thread.start()

# Keep the main thread alive
outdoor_thread.join()
indoor_thread.join()


<ipython-input-28-80d0e13ff08a>:8: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client_indoor = mqtt.Client()
<ipython-input-28-80d0e13ff08a>:14: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client_outdoor = mqtt.Client()


Indoor: Temperature: 25.17°C, Humidity: N/A%, Pressure: N/A hPa, Distance: N/A mm
Outdoor: Temperature: 28.1°C, Humidity: 51.49%, Dlight: 1240 Lux
Indoor: Temperature: 26.94°C, Humidity: 35.64%, Pressure: 976.3499 hPa, Distance: N/A mm
Outdoor: Temperature: 28.12°C, Humidity: 51.8%, Dlight: 1232 Lux
Indoor: Temperature: 27.16°C, Humidity: 35.7%, Pressure: 976.4099 hPa, Distance: N/A mm
Outdoor: Temperature: 28.08°C, Humidity: 52.19%, Dlight: 1222 Lux
Indoor: Temperature: 25.17°C, Humidity: N/A%, Pressure: N/A hPa, Distance: N/A mm
Outdoor: Temperature: 28.1°C, Humidity: 52.21%, Dlight: 1231 Lux
Indoor: Temperature: 27.17°C, Humidity: 35.46%, Pressure: 976.34 hPa, Distance: N/A mm
Outdoor: Temperature: 27.95°C, Humidity: 52.5%, Dlight: 1219 Lux
Indoor: Temperature: N/A°C, Humidity: N/A%, Pressure: N/A hPa, Distance: 707.7 mm
Outdoor: Temperature: 28.45°C, Humidity: 49.24%, Dlight: 1267 Lux
Indoor: Temperature: 27.15°C, Humidity: 35.31%, Pressure: 976.38 hPa, Distance: N/A mm
Outdoor: Te

#Generate fake sensor data

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Define the duration of the data (e.g., 24 hours)
duration_hours = 24
num_minutes = duration_hours * 60

# Define a starting timestamp
start_time = datetime.now().replace(second=0, microsecond=0)

# Generate timestamps for each minute
timestamps = [start_time + timedelta(minutes=i) for i in range(num_minutes)]

# Define realistic ranges for sensor data
ranges = {
    'Outdoor Dlight': (0, 900), # 0-900 for light levels
    'Outdoor Temperature': (10, 35), # Realistic outdoor temperature range
    'Outdoor Humidity': (30, 90),  # Realistic outdoor humidity range
    'Indoor Humidity': (30, 70),   # Realistic indoor humidity range
    'Indoor Temperature': (20, 28), # More stable indoor temperature
    'Indoor Pressure': (980, 1020), # Atmospheric pressure range
    'Indoor Distance': (400, 600)  # Plausible range for a distance sensor
}

# Generate fictional data with some variation
data = {
    'Timestamp': timestamps,
    'Outdoor Dlight': np.random.uniform(ranges['Outdoor Dlight'][0], ranges['Outdoor Dlight'][1], num_minutes),
    'Outdoor Temperature': np.random.uniform(ranges['Outdoor Temperature'][0], ranges['Outdoor Temperature'][1], num_minutes),
    'Outdoor Humidity': np.random.uniform(ranges['Outdoor Humidity'][0], ranges['Outdoor Humidity'][1], num_minutes),
    'Indoor Humidity': np.random.uniform(ranges['Indoor Humidity'][0], ranges['Indoor Humidity'][1], num_minutes),
    'Indoor Temperature': np.random.uniform(ranges['Indoor Temperature'][0], ranges['Indoor Temperature'][1], num_minutes),
    'Indoor Pressure': np.random.uniform(ranges['Indoor Pressure'][0], ranges['Indoor Pressure'][1], num_minutes),
    'Indoor Distance': np.random.uniform(ranges['Indoor Distance'][0], ranges['Indoor Distance'][1], num_minutes)
}

# Introduce some basic daily trends (optional, making it slightly more realistic)
# For example, temperature rises during the day and falls at night, Dlight follows a sun pattern.
time_of_day = (np.array([t.hour for t in timestamps]) + np.array([t.minute for t in timestamps])/60) # Time in hours

# Simple sine wave for temperature (peaks around midday)
data['Outdoor Temperature'] = data['Outdoor Temperature'] + 5 * np.sin(2 * np.pi * (time_of_day - 10)/24)
data['Indoor Temperature'] = data['Indoor Temperature'] + 1 * np.sin(2 * np.pi * (time_of_day - 12)/24)

# Simple sine wave for Dlight (peaks around midday, stays low at night)
data['Outdoor Dlight'] = data['Outdoor Dlight'] * (0.5 + 0.5 * np.sin(2 * np.pi * (time_of_day - 6)/24))
data['Outdoor Dlight'][data['Outdoor Dlight'] < 0] = 0 # Ensure Dlight is not negative

# Simple inverse sine wave for outdoor humidity (dips around midday)
data['Outdoor Humidity'] = data['Outdoor Humidity'] - 10 * np.sin(2 * np.pi * (time_of_day - 10)/24)
data['Outdoor Humidity'][data['Outdoor Humidity'] > 100] = 100 # Cap humidity at 100
data['Outdoor Humidity'][data['Outdoor Humidity'] < 0] = 0 # Ensure humidity is not negative


# Create a pandas DataFrame
df = pd.DataFrame(data)

# Round the data to a few decimal places for cleaner look
df = df.round(2)

# Define the filename
excel_filename = 'sensor_data.xlsx'

# Save the DataFrame to an Excel file
df.to_excel(excel_filename, index=False)

print(f"קובץ האקסל '{excel_filename}' נוצר בהצלחה עם {num_minutes} שורות של נתונים פיקטיביים.")

קובץ האקסל 'sensor_data.xlsx' נוצר בהצלחה עם 1440 שורות של נתונים פיקטיביים.


In [ ]:
from firebase import firebase

FBconn = firebase.FirebaseApplication(DBLink, None)

In [ ]:
# Read the Excel file into a pandas DataFrame
df = pd.read_excel('sensor_data.xlsx')

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    # Use the Timestamp as the key for Firebase
    timestamp_str = row['Timestamp'].strftime('%Y-%m-%d %H:%M:%S') # Format timestamp as string

    # Create a dictionary with the sensor data for this timestamp
    sensor_data = {
        'Outdoor Dlight': row['Outdoor Dlight'],
        'Outdoor Temperature': row['Outdoor Temperature'],
        'Outdoor Humidity': row['Outdoor Humidity'],
        'Indoor Humidity': row['Indoor Humidity'],
        'Indoor Temperature': row['Indoor Temperature'],
        'Indoor Pressure': row['Indoor Pressure'],
        'Indoor Distance': row['Indoor Distance']
    }

    # Save the data to Firebase with the timestamp as the key
    FBconn.put('/sensor_readings', timestamp_str, sensor_data)

    print(f"Saved data for timestamp: {timestamp_str}")

# Statistics window

In [4]:
!pip install requests beautifulsoup4
!pip install firebase
!pip install -q gradio
!pip install -q matplotlib

In [5]:
from firebase import firebase

FBconn = firebase.FirebaseApplication(DBLink, None)

In [27]:
import gradio as gr
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta

# Function to fetch data from Firebase for a specific time range
def fetch_data_from_firebase(full_start_time_str):
    try:
        start_time = datetime.strptime(full_start_time_str, '%Y-%m-%d %H:%M:%S')
    except ValueError as e:
        print(f"Error parsing start time string '{full_start_time_str}': {e}")
        return pd.DataFrame()

    end_time = start_time + timedelta(hours=1)

    all_data_original = FBconn.get('/sensor_readings', None)

    all_data = {}
    # marge indoor and outdoor dict
    for d in (all_data_original['indoor'], all_data_original['outdoor']):
        for key, value in d.items():
            if key not in all_data:
                all_data[key] = {}
            for k,v in value.items():
              if v == 'N/A':
                all_data[key][k] = None
              else:
                all_data[key][k] = v

    if not all_data:
        return pd.DataFrame()

    filtered_data = {}
    for timestamp_str, data in all_data.items():
        try:
            timestamp = datetime.strptime(timestamp_str, '%Y-%m-%d %H:%M:%S')
            if start_time <= timestamp < end_time:
                filtered_data[timestamp] = data
        except ValueError:
            print(f"Skipping entry with invalid timestamp in Firebase: {timestamp_str}")
            pass

    if not filtered_data:
        return pd.DataFrame()

    df = pd.DataFrame.from_dict(filtered_data, orient='index')
    if not df.empty: # Ensure index is DatetimeIndex for plotting if df is not empty
        df.index = pd.to_datetime(df.index)
        df.index.name = 'Timestamp'
        df = df.sort_index()
    return df

# Helper function to create plots (reduces redundancy)
def create_sensor_plot(df, column_name, title, ylabel):
    plt.figure(figsize=(10, 4))
    if column_name in df.columns and not df[column_name].empty:
        plt.plot(df.index, df[column_name])
    else:
        plt.text(0.5, 0.5, f'No data for {column_name}', horizontalalignment='center', verticalalignment='center')
    plt.grid(True) # Add grid
    plt.title(title)
    plt.xlabel('Timestamp')
    plt.ylabel(ylabel)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plot_fig = plt.gcf()
    plt.close()
    return plot_fig

# Function to generate plots for the selected time range
def generate_plots(selected_date_str, selected_hour_str):
    if not selected_date_str or not selected_hour_str:
        print("Date or hour not selected.")
        # Return Nones for all 7 plot outputs
        return None, None, None, None, None, None, None

    try:
        # selected_date_str is already "YYYY-MM-DD"
        selected_hour_int = int(selected_hour_str)
        full_start_time_str = f"{selected_date_str} {selected_hour_int:02d}:00:00"
    except ValueError:
        print(f"Invalid hour format: {selected_hour_str}")
        return None, None, None, None, None, None, None

    df_hour = fetch_data_from_firebase(full_start_time_str)

    if df_hour.empty:
        print(f"No data found for {full_start_time_str}")
        return None, None, None, None, None, None, None

    plot_dlight = create_sensor_plot(df_hour, 'Outdoor Dlight', 'Outdoor Dlight', 'Dlight (Lux)')
    plot_out_temp = create_sensor_plot(df_hour, 'Outdoor Temperature', 'Outdoor Temperature', 'Temp (°C)')
    plot_out_humidity = create_sensor_plot(df_hour, 'Outdoor Humidity', 'Outdoor Humidity', 'Humidity (%)')
    plot_in_humidity = create_sensor_plot(df_hour, 'Indoor Humidity', 'Indoor Humidity', 'Humidity (%)')
    plot_in_temp = create_sensor_plot(df_hour, 'Indoor Temperature', 'Indoor Temperature', 'Temp (°C)')
    plot_pressure = create_sensor_plot(df_hour, 'Indoor Pressure', 'Indoor Pressure', 'Pressure (hPa)')
    plot_distance = create_sensor_plot(df_hour, 'Indoor Distance', 'Indoor Distance', 'Distance (mm)')

    return plot_dlight, plot_out_temp, plot_out_humidity, plot_in_humidity, plot_in_temp, plot_pressure, plot_distance

# --- Data Preparation for Dropdowns ---
all_data_keys_original = FBconn.get('/sensor_readings', None) #

all_data_keys = {}
# marge indoor and outdoor dict
for d in (all_data_keys_original['indoor'], all_data_keys_original['outdoor']):
    for key, value in d.items():
        if key not in all_data_keys:
            all_data_keys[key] = {}
        for k,v in value.items():
          if v == 'N/A':
            all_data_keys[key][k] = None
          else:
            all_data_keys[key][k] = v

if all_data_keys:
    available_timestamps_str = sorted(all_data_keys.keys())
else:
    available_timestamps_str = []


# Get unique dates as STRINGS
unique_date_strings = sorted(list(set([
    datetime.strptime(ts, '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d')
    for ts in available_timestamps_str
])))

#ToDo: Add refresh button to be able to see new datetime if running too long    ##############################################

# Create mapping from STRING date to list of hours (as strings or ints)
date_to_hours_map = {}
for ts_str in available_timestamps_str:
    dt_obj = datetime.strptime(ts_str, '%Y-%m-%d %H:%M:%S')
    date_key_str = dt_obj.strftime('%Y-%m-%d')
    hour_val = dt_obj.hour # Keep as int for now, or str(dt_obj.hour)
    if date_key_str not in date_to_hours_map:
        date_to_hours_map[date_key_str] = []
    if hour_val not in date_to_hours_map[date_key_str]: # Avoid duplicate hours
        date_to_hours_map[date_key_str].append(hour_val)

for date_key_str in date_to_hours_map:
    date_to_hours_map[date_key_str].sort()
    # Convert hours to string if you prefer string choices for hour dropdown too
    date_to_hours_map[date_key_str] = [str(h) for h in date_to_hours_map[date_key_str]]


# --- Gradio Interface ---
with gr.Blocks() as demo:
  gr.Markdown("## Sensor Data Statistics")

  if unique_date_strings:
      # Initial values for dropdowns
      initial_selected_date_str = unique_date_strings[0]
      initial_hour_choices = date_to_hours_map.get(initial_selected_date_str, [])
      initial_selected_hour_str = initial_hour_choices[0] if initial_hour_choices else None

      date_dropdown = gr.Dropdown(
          choices=unique_date_strings, # STRING choices
          value=initial_selected_date_str, # STRING value
          label="Select Date"
      )

      time_dropdown = gr.Dropdown(
          choices=initial_hour_choices, # Hour choices (can be string or int)
          value=initial_selected_hour_str, # Initial hour value
          label="Select Start Hour for 1 Hour Interval"
      )

      #ToDo: Maybe add a option to choose which gragh is the main, and show it first bigger

      with gr.Row():
          plot_dlight_out = gr.Plot(label="Outdoor Dlight")
          plot_temp_out = gr.Plot(label="Outdoor Temperature")
      with gr.Row():
          plot_humidity_out = gr.Plot(label="Outdoor Humidity")
          plot_humidity_in = gr.Plot(label="Indoor Humidity")
      with gr.Row():
          plot_temp_in = gr.Plot(label="Indoor Temperature")
          plot_pressure_in = gr.Plot(label="Indoor Pressure")
      with gr.Row():
          plot_distance_in = gr.Plot(label="Indoor Distance")

      plot_outputs_list = [
          plot_dlight_out, plot_temp_out, plot_humidity_out,
          plot_humidity_in, plot_temp_in, plot_pressure_in, plot_distance_in
      ]

      def update_hour_dropdown_and_plots(selected_date_str_event):
          # This function is triggered by date_dropdown change.
          # It needs to update hour_dropdown's choices AND trigger plot regeneration.
          new_hour_choices = date_to_hours_map.get(selected_date_str_event, [])
          new_selected_hour = new_hour_choices[0] if new_hour_choices else None

          # Update plots based on the new date and the first available hour
          plot_updates = generate_plots(selected_date_str_event, new_selected_hour)

          # Return updates for time_dropdown and all plot outputs
          return [gr.update(choices=new_hour_choices, value=new_selected_hour)] + list(plot_updates if plot_updates else [None]*7)

      date_dropdown.change(
          fn=update_hour_dropdown_and_plots,#update_hour_dropdown_and_plots,
          inputs=date_dropdown, # Passes selected string date
          outputs=[time_dropdown] + plot_outputs_list # Update time_dropdown AND all plots
      )

      time_dropdown.change(
          fn=generate_plots, # Already expects string date and string hour
          inputs=[date_dropdown, time_dropdown],
          outputs=plot_outputs_list # Correct: output to the plot components
      )

      # Initial plot generation on load
      demo.load(
          fn=generate_plots,
          inputs=[date_dropdown, time_dropdown], # Will use initial string values
          outputs=plot_outputs_list
      )
  else:
      gr.Markdown("No data available to determine dates from Firebase.")

demo.launch(debug=False)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6336e6d37f1b57af13.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
